In [1]:
pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 113.4 MB/s eta 0:00:00


In [4]:
!yt-dlp \
  -x --audio-format wav \
  --audio-quality 0 \
  -o "raw_audio/%(playlist_index)s_%(title)s.%(ext)s" \
  "https://youtube.com/playlist?list=PLDIoUOhQQPlWm_njQtKkNIk5RYSGgzomm&si=YQHK8N6jOrwphLZD"

[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLDIoUOhQQPlWm_njQtKkNIk5RYSGgzomm&si=YQHK8N6jOrwphLZD
[youtube:tab] PLDIoUOhQQPlWm_njQtKkNIk5RYSGgzomm: Downloading webpage
[youtube:tab] PLDIoUOhQQPlWm_njQtKkNIk5RYSGgzomm: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Songs 2026 - Songs with Lyrics Playlist - My Mix - Mix Songs - Music Playlist 2026
[youtube:tab] PLDIoUOhQQPlWm_njQtKkNIk5RYSGgzomm page 1: Downloading API JSON
[youtube:tab] Playlist Songs 2026 - Songs with Lyrics Playlist - My Mix - Mix Songs - Music Playlist 2026: Downloading 100 items of 100
[download] Downloading item 1 of 100
[youtube] Extracting URL: https://www.youtube.com/watch?v=htk6MRjmcnQ
[youtube] htk6MRjmcnQ: Downloading webpage
[youtube] htk6MRjmcnQ: Downloading android sdkless player API JSON
[youtube] htk6MRjmcnQ: Downloading web safari player API JSON
[youtube] htk6MRjmcnQ: Downloading m3u8 information
[info] htk6MRjmcnQ: Downloading 1 format(s):

In [5]:
import librosa
import soundfile as sf
import os

INPUT_DIR = "raw_audio"
OUTPUT_DIR = "clips"

SR = 16000
SKIP_START = 20      # seconds to skip from beginning
CLIP_DURATION = 20  # seconds per clip

os.makedirs(OUTPUT_DIR, exist_ok=True)

for file in os.listdir(INPUT_DIR):
    if not file.endswith(".wav"):
        continue

    path = os.path.join(INPUT_DIR, file)
    y, sr = librosa.load(path, sr=SR)

    total_duration = len(y) / sr

    # If song is too short, skip
    if total_duration <= SKIP_START + CLIP_DURATION:
        continue

    start_sample = int(SKIP_START * sr)
    clip_samples = int(CLIP_DURATION * sr)

    clip_idx = 0
    current_pos = start_sample

    while current_pos + clip_samples <= len(y):
        clip = y[current_pos : current_pos + clip_samples]

        out_name = f"{os.path.splitext(file)[0]}_clip{clip_idx}.wav"
        out_path = os.path.join(OUTPUT_DIR, out_name)

        sf.write(out_path, clip, sr)

        clip_idx += 1
        current_pos += clip_samples  # non-overlapping


In [8]:
!zip -r clips.zip clips

  adding: clips/ (stored 0%)
  adding: clips/017_ROSÉ & Bruno Mars - APT. (Lyrics)_clip6.wav (deflated 1%)
  adding: clips/074_Miley Cyrus - Flowers (Lyrics)_clip6.wav (deflated 3%)
  adding: clips/019_Daydream - Jason Derlatka (Official Music Video)_clip0.wav (deflated 2%)
  adding: clips/097_Mariah the Scientist, Kali Uchis - Is It a Crime (Official Music Video)_clip2.wav (deflated 2%)
  adding: clips/020_Billie Eilish - BIRDS OF A FEATHER (Official Lyric Video)_clip3.wav (deflated 2%)
  adding: clips/075_Eminem - Mockingbird (Lyrics)_clip7.wav (deflated 2%)
  adding: clips/018_Gracie Abrams - That’s So True (Lyrics)_clip5.wav (deflated 2%)
  adding: clips/052_Clean Bandit - Symphony (Lyrics) Feat. Zara Larsson_clip4.wav (deflated 4%)
  adding: clips/060_Tate McRae - It's ok I'm ok (Lyrics)_clip0.wav (deflated 1%)
  adding: clips/011_Alphaville - Forever Young (Lyrics)_clip6.wav (deflated 4%)
  adding: clips/042_Chappell Roan - HOT TO GO! (Lyrics)_clip0.wav (deflated 3%)
  adding: cl

In [9]:
pip install -q openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 46.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
import whisper
import os

AUDIO_DIR = "clips"
LYRICS_DIR = "lyrics"

os.makedirs(LYRICS_DIR, exist_ok=True)

# Choose model size
model = whisper.load_model("small")
# use "base" if GPU memory is low

for file in os.listdir(AUDIO_DIR):
    if not file.endswith(".wav"):
        continue

    audio_path = os.path.join(AUDIO_DIR, file)
    print(f"Transcribing: {file}")

    result = model.transcribe(audio_path)
    text = result["text"].strip()

    out_file = file.replace(".wav", ".txt")
    out_path = os.path.join(LYRICS_DIR, out_file)

    with open(out_path, "w", encoding="utf-8") as f:
        f.write(text)


100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 193MiB/s]


Transcribing: 017_ROSÉ & Bruno Mars - APT. (Lyrics)_clip6.wav
Transcribing: 074_Miley Cyrus - Flowers (Lyrics)_clip6.wav
Transcribing: 019_Daydream - Jason Derlatka (Official Music Video)_clip0.wav
Transcribing: 097_Mariah the Scientist, Kali Uchis - Is It a Crime (Official Music Video)_clip2.wav
Transcribing: 020_Billie Eilish - BIRDS OF A FEATHER (Official Lyric Video)_clip3.wav
Transcribing: 075_Eminem - Mockingbird (Lyrics)_clip7.wav
Transcribing: 018_Gracie Abrams - That’s So True (Lyrics)_clip5.wav
Transcribing: 052_Clean Bandit - Symphony (Lyrics) Feat. Zara Larsson_clip4.wav
Transcribing: 060_Tate McRae - It's ok I'm ok (Lyrics)_clip0.wav
Transcribing: 011_Alphaville - Forever Young (Lyrics)_clip6.wav
Transcribing: 042_Chappell Roan - HOT TO GO! (Lyrics)_clip0.wav
Transcribing: 041_Lady Gaga - The Dead Dance (Lyrics)_clip1.wav
Transcribing: 086_Rex Orange County - THE SHADE (Lyrics)_clip7.wav
Transcribing: 008_yung kai - blue (Lyrics)_clip8.wav
Transcribing: 038_Justin Bieber -

In [11]:
!zip -r lyrics.zip lyrics

  adding: lyrics/ (stored 0%)
  adding: lyrics/023_Rock That Body - The Black Eyed Peas (Lyrics)_clip4.txt (deflated 49%)
  adding: lyrics/084_Meghan Trainor - No (Lyrics) Untouchable_clip1.txt (deflated 37%)
  adding: lyrics/024_Eddie Benjamin - KISS ME (Official Music Video)_clip7.txt (deflated 8%)
  adding: lyrics/047_Ed Sheeran - Sapphire (Lyrics)_clip5.txt (stored 0%)
  adding: lyrics/073_Tiësto - 10_35 (Lyrics) ft. Tate McRae_clip0.txt (deflated 34%)
  adding: lyrics/002_Lady Gaga, Bruno Mars - Die With A Smile (Lyrics)_clip0.txt (deflated 23%)
  adding: lyrics/075_Eminem - Mockingbird (Lyrics)_clip4.txt (deflated 40%)
  adding: lyrics/020_Billie Eilish - BIRDS OF A FEATHER (Official Lyric Video)_clip4.txt (deflated 24%)
  adding: lyrics/079_Doja Cat - Paint The Town Red (Lyrics)_clip9.txt (deflated 28%)
  adding: lyrics/029_BLACKPINK '뛰어(JUMP)' (Color Coded Lyrics)_clip2.txt (deflated 37%)
  adding: lyrics/092_The Chainsmokers & Coldplay - Something Just Like This (Official Lyri

In [12]:
from google.colab import files

files.download('lyrics.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>